# LangChain 学习手册 03：工具、记忆与 Agent

本节基于 LangChain v0.3.79，介绍如何把检索、工具调用与 Agent 框架结合起来。


## 准备检索工具
你可以选择任何向量数据库或文档检索方案。下面示例用 `InMemoryVectorStore` 结合 `OpenAIEmbeddings`，演示如何构建简单的检索链。


In [ ]:
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

texts = [
    'LangChain 通过模块化组件帮助开发者构建多种 LLM 应用。',
    'LangChain 的 Runnable 管线让你可以灵活组合提示、模型与工具。',
    'LangChain 与多种向量数据库、检索器与代理框架集成。'
]

splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)

split_docs = [
    Document(page_content=chunk)
    for text in texts
    for chunk in splitter.split_text(text)
]

vectorstore = InMemoryVectorStore.from_documents(
    split_docs,
    OpenAIEmbeddings(model='text-embedding-3-small')
)
retriever = vectorstore.as_retriever()


> ℹ️ 如果没有 OpenAI API Key，可以将 `OpenAIEmbeddings` 替换为社区提供的本地向量模型，或在演示时使用 `FakeEmbeddings`。


## 构建检索增强链 (RAG)
借助 `RunnableParallel` 与 `ChatPromptTemplate`，我们可以快速搭建一个检索增强的回答流程。


In [ ]:
            from langchain_core.runnables import RunnableParallel, RunnableLambda
            from langchain_core.prompts import ChatPromptTemplate
            from langchain_community.chat_models.fake import FakeListChatModel
            from langchain_core.output_parsers import StrOutputParser

            model = FakeListChatModel(
                responses=['根据检索内容，建议先阅读 LangChain 官方文档，再结合示例实践。']
            )

            prompt = ChatPromptTemplate.from_messages(
                [
                    ('system', '你是一位 LangChain 学习顾问，参考上下文回答问题。'),
                    ('human', '问题: {question}
上下文: {context}')
                ]
            )

            rag_chain = (
                RunnableParallel(
                    context=retriever,
                    question=RunnableLambda(lambda x: x['question'])
                )
                | prompt
                | model
                | StrOutputParser()
            )

            print(rag_chain.invoke({'question': '学习 LangChain 的路线是什么？'}))


## 构建 ReAct Agent
LangChain 0.3.79 提供 `create_react_agent` 等辅助函数，可快速让大模型调用工具。


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

@tool
def recommend_resource(topic: str) -> str:
    """基于主题返回推荐资源。"""
    if 'langchain' in topic.lower():
        return '推荐阅读 LangChain 官方文档与 Cookbook。'
    return '请指定更明确的主题。'

# 若无 API Key，可改用 Fake 模型并模拟输出。
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

agent = create_react_agent(llm, [recommend_resource])
executor = AgentExecutor(agent=agent, tools=[recommend_resource], verbose=True)

result = executor.invoke({'input': '帮我推荐 LangChain 的学习资料'})
result['output']


> ⚠️ ReAct Agent 通常需要真实的大模型能力。如果没有网络或 API Key，可以阅读输出日志并把 `ChatOpenAI` 替换为能离线运行的模型。


## 下一步
- 深入阅读官方文档的 [How-to Guides](https://python.langchain.com/docs/how_to/) 与 [API 参考](https://api.python.langchain.com/en/latest/)。
- 尝试把 LangChain 与你熟悉的框架（FastAPI、Streamlit 等）结合。
- 持续关注 v0.3.x 的更新日志，及时调整代码结构。
